# Notebook App for Data Enrichment Using LangGrpah and Tavily

## Introduction
Welcome to the Data Enrichment Sheet!

This sheet is designed to enhance your data by populating missing values using advanced search capabilities from Tavily and the LangGraph Framework. It accepts either a CSV or Excel file path or a Google Sheet name (follow the instructions in the "Google Colab Set Up" section).

For Google Sheets and Excel files, the agent will create an additional sheet containing the populated data table. For CSV files, a new CSV file will be generated with the enriched data provided by the agent.

For detailed instructions on how to use this sheet and call the agent to start enriching your data, please refer to the "Call Agent" section below.

Note: The results generated by the DataEnrichmentAgent are not perfect and could be improved.

## Requirements


In [ ]:
!pip install langgraph tavily-python openai openpyxl python-dotenv

## Libraries

In [1]:
import pandas as pd
import numpy as np
import json
import asyncio
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from typing import TypedDict, Annotated
from openai import AsyncOpenAI
from tavily import AsyncTavilyClient
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage

## Google Colab Set Up

To connent with Google Sheets upload this notebook to Google Colab. Uncomment the following lines and ensure to input your Google Sheet name for your agent in the Call Agent section.

In [15]:
# import gspread # Python API for Google Sheets
# from google.colab import auth
# from google.auth import default

# # Authenticate and create the gspread client
# auth.authenticate_user()
# creds, _ = default()

# gc = gspread.authorize(creds)

## Build Master Agent

### Master's Agent State

In [2]:
class AgentState(TypedDict):
  raw_data: pd.DataFrame                  # original panda data frame
  new_data: pd.DataFrame                  # populated panda data frame
  colab: str                              # google sheet name
  excel: str                              # path to Excel file
  csv: str                                # path to CSV file

### Master's Agent class

In [3]:
class DataEnrichmentAgent():
  def __init__(self):
    # Initialize agents
    data_agent = DataAgent()
    enrich_agent = EnrichAgent()

    workflow = StateGraph(AgentState)

    workflow.add_node("get_data", data_agent.get_df)
    workflow.add_node("enrich_data", enrich_agent.run)
    workflow.add_node("write_data", data_agent.write_df)

    workflow.set_entry_point("get_data")

    workflow.add_edge("get_data", "enrich_data")
    workflow.add_edge("enrich_data", "write_data")
    workflow.add_edge("write_data", END)

    self.workflow = workflow.compile()

## Sub Agents

### Data Agent

In [21]:
class DataAgent():
  def __init__(self):
    pass
  
  # Convert columns with numeric values to appropriate types
  def convert_numeric_columns(self, df):
      for col in df.columns:
          # Check if column contains numeric values
          if pd.api.types.is_numeric_dtype(df[col]):
              # Attempt to convert to integers if possible
              try:
                  df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
              except ValueError:
                  # If conversion to Int64 fails, keep as float
                  df[col] = pd.to_numeric(df[col], errors='coerce')
      return df

  # Google Sheets Integration through Colab
  def connect_to_google_sheets_colab(self, sheet_name):
    sheet = gc.open(sheet_name).sheet1
    data = sheet.get_all_records()
    df = pd.DataFrame(data)
    df.replace('', np.nan, inplace=True)
    return df

  def get_df(self, state: AgentState):
    # Function to get Data Frame and store in state
    if state.get('colab'):
        df = self.connect_to_google_sheets_colab(state['colab'])
        df = self.convert_numeric_columns(df)
        df = df.astype(str)  # Ensure all values are strings
        return {"raw_data": df}
    elif state.get('csv'):
        df = pd.read_csv(state['csv'])
        return {"raw_data": df}
    elif state.get('excel'):
        df = pd.read_excel(state['excel'])
        return {"raw_data": df}
    else:
        print("Either colab or csv or excel argument is required.")
        return

  def write_df(self, state: AgentState):
    try:
      df = state['new_data']
      if state['colab']:
          # Use existing Colab authentication (assuming `gc` is a gspread client)
          spreadsheet = gc.open(state['colab'])

          # Add a new sheet
          sheet_name = f"AgentSheet{len(spreadsheet.worksheets()) + 1}"
          spreadsheet.add_worksheet(title=sheet_name, rows=str(df.shape[0]), cols=str(df.shape[1]))
          new_sheet = spreadsheet.worksheet(sheet_name)

          # Write DataFrame to the new sheet
          new_sheet.update([df.columns.values.tolist()] + df.values.tolist())
          print(f"Data written to new sheet '{sheet_name}' in Google Sheets document '{state['colab']}'.")

      elif state['excel']:
        # Load the existing workbook
        wb = load_workbook(state['excel'])

        # Generate a new sheet name based on existing sheets count
        sheet_name = f"AgentSheet{len(wb.sheetnames) + 1}"

        # Create a new sheet and write data
        ws = wb.create_sheet(title=sheet_name)

        # Write DataFrame to the new sheet
        for r in dataframe_to_rows(df, index=False, header=True):
          ws.append(r)

        # Save changes to the Excel file
        wb.save(state['excel'])
        wb.close()
        print(f"Data written to new sheet '{sheet_name}' in Excel document '{state['excel']}'.")

      elif state['csv']:
        # Extract file name from the csv path and construct new name
        file_name = os.path.basename(state['csv'])
        # Output DataFrame to a CSV file
        output_csv = f"AgentSheet{file_name}" # construct new file path
        df.to_csv(output_csv, index=False)
        print(f"Data written to CSV file '{output_csv}'.")
      
      else:
        print("No valid file type specified.")
    except Exception as e:
      print(f"Error occurred: {e}")

### Enrich Agent

In [22]:
# Colors
RED = '\033[91m'
GREEN = '\033[92m'
BLUE = '\033[94m'
YELLOW = '\033[93m'
ENDC = '\033[0m'

In [23]:
class EnrichAgent():
  def __init__(self):
    # Maximum number of columns to try to resolve at once
    self.MAX_COLS_PER_PASS = 5

    # Maximum number of fill-in passes to attempt
    self.MAX_PASSES = 5

    self.model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
  
  def is_missing_value(self, value):
        """
        Check if a value is considered missing. Handles various representations of missing values.
        """
        if pd.isna(value) or value == '' or str(value).lower() == 'nan':
            return True
        return False

  async def generate_search_query(self, head, columns):
    """
    This function takes the head of the table (all column names) and the names of the columns that it needs to fill in, and generates a search prompt for Tavily
    """
    prompt = f"""You are a researcher with the task of filling in a spreadsheet. The spreadsheet contains the columns {str(head)} and {str(columns)} have not been filled in yet.
    Write a web search query for a search engine that you will use to fill in each row of the spreadsheet one by one. In the query, replace the name of the entry you are researching with $ENTRY
    Respond only with the query."""

    messages = [SystemMessage(content=prompt)]
    response = await self.model.ainvoke(messages)

    return response.content.strip('\'"')

  async def fill_in_row(self, df, head, row_index, columns, search_query):
    """
    This function takes a data frame, a row to complete, a list of column names to complete and the query to search.
    It calls the Tavily API to retrieve information using the search_query and prompts OpenAI to extract column values from the response
    """
    # if the search_query is a coroutine, await it
    if asyncio.iscoroutine(search_query):
        search_query = await search_query

    entry = str(df.iloc[row_index][head[0]])

    search_query = search_query.replace("$ENTRY", entry)
    try:
        print(f"{BLUE}Tavily Search: {search_query}{ENDC}")
        tavily_response = await tavily.search(search_query)
        # Only save the title and content from each Tavily result
        research_results = [{'title': r['title'], 'content': r['content']} for r in tavily_response['results']]
    except Exception as e:
        print(f"{RED}Error during Tavily search: {str(e)}{ENDC}")
        return

    # To ensure consistency with the existing data, come up with an example based on existing data in the df
    example = {}
    for col in columns:
      for i in range(len(df)):
        if i != row_index and not self.is_missing_value(df.iloc[i][col]):
          example[col] = df.iloc[i][col]
          break

    example_str = f'Example: {str(example)}\n' if len(example) > 0 else ''

    prompt = f"You are filling in a spreadsheet using online sources. The following fields need to be filled in for the entry '{entry}': {', '.join(columns)}\n" \
              "Using the data below, find values for as many of these fields as you can. Reply with a JSON object, linking each field to its value.\n" \
            f"{example_str}" \
              "Values must be written in a spreadsheet friendly format. If the data is not sufficient to complete a field, simply omit it from the result object.\n" \
              "Do not incude any text other than the JSON object in your response.\n\n" \
                      f"{str(research_results)}"

    json_model = self.model.bind(response_format={"type": "json_object"})
    messages = [SystemMessage(content=prompt)]
    response = await json_model.ainvoke(messages)

    json_str = response.content
    try:
      fields = json.loads(json_str)
    except json.JSONDecodeError as e:
      print(f"{RED}Failed to parse JSON: {json_str}{ENDC}")
      return

    for field, value in fields.items():
      if field not in columns:
        print(f"{RED}LLM returned an invalid field '{field}'{ENDC}")
        continue

      col_index = df.columns.get_loc(field)
      if not self.is_missing_value(df.iloc[row_index][field]):
        print(f"{YELLOW}Skipping field '{field}' as it already has a value:{ENDC}", df.iloc[row_index][field])
        continue

      df.at[row_index, field] = str(value)
      print(f"{GREEN}Filled in field '{field}' with value '{value}'{ENDC}")

  async def run(self, state: AgentState):
    """
    Main Function:
    First, we try to fill in the maximum number of column in one pass for each row. We then loop and try to fill in the missing columns
    """
    df = state['raw_data'].copy()
    head = list(df.columns)
    # First pass: try to fill in as much as possible
    #general_query = await self.generate_search_query(head, head[1:self.MAX_COLS_PER_PASS+1])
    general_query = await self.generate_search_query(head, head[1:])

    coroutines = []
    for row_index in range(len(df)):
      has_missing_columns = False
      for col_index in range(len(head)):
        if self.is_missing_value(df.iloc[row_index][head[col_index]]):
          has_missing_columns = True
          break


      if not has_missing_columns:
        print(f"{YELLOW}No missing columns for row {row_index}{ENDC}")
        continue

      coroutines.append(self.fill_in_row(df, head, row_index, head[1:], general_query))

    await asyncio.gather(*coroutines)

    # Now, fill in the missing fields for each row
    for _ in range(self.MAX_PASSES):
      all_completed = True

      coroutines = []
      for row_index in range(len(df)):
        missing_columns = []
        for col_index in range(len(head)):
          if self.is_missing_value(df.iloc[row_index][head[col_index]]) and len(missing_columns) < self.MAX_COLS_PER_PASS:
            missing_columns.append(head[col_index])

        if len(missing_columns) > 0:
          all_completed = False
          print(f"Missing columns for row {row_index}: {', '.join(missing_columns)}")
          query = self.generate_search_query(head, missing_columns)
          coroutines.append(self.fill_in_row(df, head, row_index, missing_columns, query))
        else:
          print(f"No missing columns for row {row_index}")

      if all_completed:
        break

      await asyncio.gather(*coroutines)

    # Check if everything was completed
    all_completed = True
    for row_index in range(len(df)):
      for col_index in range(len(head)):
        if self.is_missing_value(df.iloc[row_index][head[col_index]]):
          all_completed = False
          break

    if all_completed:
      print(f"{GREEN}All rows completed{ENDC}")
    else:
      print(f"{RED}Not all rows completed{ENDC}")
    return {"new_data": df}

# Call Agent

In [24]:
import os
# Set Your API Keys
# OPENAI_API_KEY = "YOUR OPENAI API KEY"
# TAVILY_API_KEY = "YOUR TAIVLY API KEY"

# Or use .env file 
from dotenv import load_dotenv
load_dotenv('.env')
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai = AsyncOpenAI(api_key=OPENAI_API_KEY)
tavily = AsyncTavilyClient(TAVILY_API_KEY)

In [25]:
# Define the input data for your agent, specifying the path to either a CSV file or an Excel file,
# and optionally the name of your Google Sheet
# csv_path = "your_csv_path"       #E.g "Desktop/my_csv.scv"
excel_path = "data.xlsx"         #E.g This is am example Excel file that could be found in this repoo
# colab = "google_sheet_name"      #E.g "My_Google_Sheet"

In [26]:
my_agent = DataEnrichmentAgent()
input = {
    # Example: Uncomment and specify the path to your CSV file above
    # "csv": csv_path,

    # Example: Uncomment and specify the path to your Excel file above
    "excel": excel_path,

    # Example: Uncomment and specify your Google Sheet name above
    # "colab": colab
}
result = await my_agent.workflow.ainvoke(input)

Tavily Search: Movie Fast and Furious details including Genre, Release date, Director, Length, Composer, Has sequel
Tavily Search: Movie Despicable Me details including Genre, Release date, Director, Length, Composer, Has sequel
Tavily Search: Movie Titanic details including Genre, Release date, Director, Length, Composer, Has sequel
Tavily Search: Movie Avatar details including Genre, Release date, Director, Length, Composer, Has sequel
Tavily Search: Movie The Godfather details including Genre, Release date, Director, Length, Composer, Has sequel
Tavily Search: Movie Skyfall details including Genre, Release date, Director, Length, Composer, Has sequel


/tmp/ipykernel_19911/2389177379.py:91: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Romantic/Disaster' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[row_index, field] = str(value)
/tmp/ipykernel_19911/2389177379.py:91: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1997' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[row_index, field] = str(value)
/tmp/ipykernel_19911/2389177379.py:91: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'James Cameron' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[row_index, field] = str(value)


Filled in field 'Genre' with value 'Romantic/Disaster'
Filled in field 'Release date' with value '1997'
Filled in field 'Director' with value 'James Cameron'
Filled in field 'Has sequel' with value 'No'
Filled in field 'Genre' with value 'Action'
Filled in field 'Release date' with value 'June 11, 2001'
Filled in field 'Director' with value 'Rob Cohen'
Filled in field 'Has sequel' with value 'Yes'
Filled in field 'Genre' with value 'Action-Adventure, Fantasy, Science Fiction'
Filled in field 'Release date' with value 'December 18, 2009'
Filled in field 'Director' with value 'James Cameron'
Filled in field 'Length' with value '3 Hours'
Skipping field 'Has sequel' as it already has a value: Yes
Filled in field 'Genre' with value 'Gangster, Epic'
Filled in field 'Release date' with value 'March 15, 1972'
Filled in field 'Director' with value 'Francis Ford Coppola'
Filled in field 'Has sequel' with value 'Yes'
Filled in field 'Genre' with value 'Animation, Adventure, Comedy, Crime, Family,

/tmp/ipykernel_19911/2389177379.py:91: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Thomas Newman' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[row_index, field] = str(value)


Tavily Search: Length and Composer of Titanic movie
Tavily Search: Movie Avatar composer
Tavily Search: Who composed the soundtrack for Despicable Me movie?
Tavily Search: Who composed the soundtrack for Fast and Furious movie?
Tavily Search: Length and Composer of The Godfather movie
Filled in field 'Composer' with value 'James Horner'
Filled in field 'Composer' with value 'Heitor Pereira, Pharrell Williams'
Filled in field 'Composer' with value 'Brian Tyler'
Filled in field 'Length' with value '2 Hours 55 Minutes'
Filled in field 'Composer' with value 'Nino Rota'
Filled in field 'Composer' with value 'James Horner'
No missing columns for row 0
No missing columns for row 1
Missing columns for row 2: Length
No missing columns for row 3
No missing columns for row 4
No missing columns for row 5
Tavily Search: Length of Titanic movie
Filled in field 'Length' with value '3 Hours 15 Minutes'
No missing columns for row 0
No missing columns for row 1
No missing columns for row 2
No missing co

#### Your original Data

In [27]:
result['raw_data']

,Movie name,Genre,Release date,Director,Length,Composer,Has sequel
0,Fast and Furious,NaN,NaN,NaN,2 Hours 20 Minutes,NaN,NaN
1,Despicable Me,NaN,NaN,NaN,NaN,NaN,NaN
2,Titanic,NaN,NaN,NaN,NaN,NaN,NaN
3,Avatar,NaN,NaN,NaN,NaN,NaN,Yes
4,The Godfather,NaN,NaN,NaN,NaN,NaN,NaN
5,Skyfall,NaN,NaN,NaN,NaN,NaN,NaN


#### Enriched Data

In [28]:
result['new_data']

,Movie name,Genre,Release date,Director,Length,Composer,Has sequel
0,Fast and Furious,Action,"June 11, 2001",Rob Cohen,2 Hours 20 Minutes,Brian Tyler,Yes
1,Despicable Me,"Animation, Adventure, Comedy, Crime, Family, S...","Jul 9, 2010","Chris Renaud, Pierre Coffin",1 h 35 m,"Heitor Pereira, Pharrell Williams",Yes
2,Titanic,Romantic/Disaster,1997,James Cameron,3 Hours 15 Minutes,James Horner,No
3,Avatar,"Action-Adventure, Fantasy, Science Fiction","December 18, 2009",James Cameron,3 Hours,James Horner,Yes
4,The Godfather,"Gangster, Epic","March 15, 1972",Francis Ford Coppola,2 Hours 55 Minutes,Nino Rota,Yes
5,Skyfall,"Action Thriller, Spy Film","October 26, 2012",Sam Mendes,143 min,Thomas Newman,Yes


In [29]:
result['new_data'].to_excel('new_data.xlsx', index=False)